# 7.0. PySpark

In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1639082831106_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Importando as bibliotecas usadas no projeto

In [3]:
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from  pyspark.sql.functions import abs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.install_pypi_package("pandas==0.25.1") 
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("numpy")

Lendo dataset do bucket no S3

In [4]:
input_bucket = 's3://ede.miranda.bucket'
input_path = '/datasets/output/wildfire_train_spark_sprint3.csv'
df = spark.read.csv(input_bucket + input_path, header=True)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-------------------+-------------------+--------------------------+----------------+--------+-----------+------------+--------------+--------------+------------------+--------------+--------------+-------------+--------------------+-----------+--------------+---------+
|days_no_rain|precipitation|           latitude|          longitude|atm_pressure_station_level|global_radiation|air_temp|dpoint_temp|air_humidity|wind_clockwise|max_gust_speed|        day_length|biome_Amazonia|biome_Caatinga|biome_Cerrado|biome_Mata Atlantica|biome_Pampa|biome_Pantanal|fire_risk|
+------------+-------------+-------------------+-------------------+--------------------------+----------------+--------+-----------+------------+--------------+--------------+------------------+--------------+--------------+-------------+--------------------+-----------+--------------+---------+
|         0.0|          0.1|             -2.842|            -41.986|                    1006.7|          2

Removendo instâncias com valores nulos

In [5]:
df = df.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('days_no_rain', 'string'), ('precipitation', 'string'), ('latitude', 'string'), ('longitude', 'string'), ('atm_pressure_station_level', 'string'), ('global_radiation', 'string'), ('air_temp', 'string'), ('dpoint_temp', 'string'), ('air_humidity', 'string'), ('wind_clockwise', 'string'), ('max_gust_speed', 'string'), ('day_length', 'string'), ('biome_Amazonia', 'string'), ('biome_Caatinga', 'string'), ('biome_Cerrado', 'string'), ('biome_Mata Atlantica', 'string'), ('biome_Pampa', 'string'), ('biome_Pantanal', 'string'), ('fire_risk', 'string')]

In [7]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['days_no_rain', 'precipitation', 'latitude', 'longitude', 'atm_pressure_station_level', 'global_radiation', 'air_temp', 'dpoint_temp', 'air_humidity', 'wind_clockwise', 'max_gust_speed', 'day_length', 'biome_Amazonia', 'biome_Caatinga', 'biome_Cerrado', 'biome_Mata Atlantica', 'biome_Pampa', 'biome_Pantanal', 'fire_risk']

Convertendo os tipos de dados para float, e modularizando os valores de latitude e longitude

In [8]:
for c in df.columns:
    df=df.withColumn(c, df[c].cast('float'))
df = df.withColumn('latitude', abs(df.latitude))
df = df.withColumn('longitude', abs(df.longitude))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('days_no_rain', 'float'), ('precipitation', 'float'), ('latitude', 'float'), ('longitude', 'float'), ('atm_pressure_station_level', 'float'), ('global_radiation', 'float'), ('air_temp', 'float'), ('dpoint_temp', 'float'), ('air_humidity', 'float'), ('wind_clockwise', 'float'), ('max_gust_speed', 'float'), ('day_length', 'float'), ('biome_Amazonia', 'float'), ('biome_Caatinga', 'float'), ('biome_Cerrado', 'float'), ('biome_Mata Atlantica', 'float'), ('biome_Pampa', 'float'), ('biome_Pantanal', 'float'), ('fire_risk', 'float')]

Renomeando a feature dependente para 'label'

In [10]:
df = df.withColumnRenamed('fire_risk', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+---------+--------------------------+----------------+--------+-----------+------------+--------------+--------------+----------+--------------+--------------+-------------+--------------------+-----------+--------------+-----+
|days_no_rain|precipitation|latitude|longitude|atm_pressure_station_level|global_radiation|air_temp|dpoint_temp|air_humidity|wind_clockwise|max_gust_speed|day_length|biome_Amazonia|biome_Caatinga|biome_Cerrado|biome_Mata Atlantica|biome_Pampa|biome_Pantanal|label|
+------------+-------------+--------+---------+--------------------------+----------------+--------+-----------+------------+--------------+--------------+----------+--------------+--------------+-------------+--------------------+-----------+--------------+-----+
|         0.0|          0.1|   2.842|   41.986|                    1006.7|          2318.2|    31.6|       21.3|        55.0|          88.0|          10.6| 12.161663|           0.0|           0.0|         

Dividindo o dataset entre treinamento e teste

In [12]:
training, test = df.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Criando VectorAssembler.

Todos os recursos de entrada devem estar em uma coluna antes de alimentar o modelo

In [13]:

assembler = VectorAssembler(inputCols=['days_no_rain', 'precipitation', 'latitude', 'longitude',
                                       'atm_pressure_station_level', 'global_radiation', 'air_temp', 
                                       'dpoint_temp', 'air_humidity', 'wind_clockwise', 'max_gust_speed', 
                                       'day_length', 'biome_Amazonia', 'biome_Caatinga', 'biome_Cerrado', 
                                       'biome_Mata Atlantica', 'biome_Pampa', 'biome_Pantanal'], \
                            outputCol="inputFeatures")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Normalizando os dados

In [14]:
scaler = Normalizer(inputCol="inputFeatures", outputCol="features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Definindo os modelos a serem utilizados e definindo so pipelines

In [23]:
lr=LinearRegression()
rfr = RandomForestRegressor()
dtr = DecisionTreeRegressor()


pipeline1=Pipeline(stages=[assembler, scaler, lr])
pipeline2=Pipeline(stages=[assembler, scaler, rfr])
pipeline3=Pipeline(stages=[assembler, scaler, dtr])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 7.1. LinearRegression

In [22]:
#Definindo os parametros a serem otimizados
paramgrid1 =ParamGridBuilder()\
.addGrid(lr.regParam, [0.0, 0.1])\
.build()


#Definindo 'rmse' como metrica de avaliação
evaluator=RegressionEvaluator(metricName="rmse")

#Definindo os parametros de crossvalidation
crossval = CrossValidator(estimator=pipeline1,  
                         estimatorParamMaps=paramgrid1,
                         evaluator = evaluator, 
                         numFolds=10
                        )
#Treinando o modelo e exbindo o score
cvModel1=crossval.fit(training) 
linear_reg = evaluator.evaluate(cvModel1.transform(test))
linear_reg

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.26657931014085273

## 7.3. RandomForestRegression

In [27]:
#Definindo os parametros a serem otimizados
paramgrid2=ParamGridBuilder()\
.addGrid(rfr.numTrees, [20, 30])\
.addGrid(rfr.maxDepth, [5, 10])\
.build()

#Definindo 'rmse' como metrica de avaliação
evaluator=RegressionEvaluator(metricName="rmse")

#Definindo os parametros de crossvalidation
crossval= CrossValidator(estimator=pipeline2,  
                         estimatorParamMaps=paramgrid2,
                         evaluator = evaluator , 
                         numFolds=10
                        )
#Treinando o modelo e exbindo o score
cvModel2=crossval.fit(training) 
r_forest_reg = evaluator.evaluate(cvModel2.transform(test))
r_forest_reg

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.20084046235790612

## 7.4. DecisonTreeRegressor

In [26]:
#Definindo os parametros a serem otimizados
paramgrid3 =ParamGridBuilder()\
.addGrid(rfr.maxDepth, [5, 10])\
.build()

#Definindo 'rmse' como metrica de avaliação
evaluator=RegressionEvaluator(metricName="rmse")

#Definindo os parametros de crossvalidation
crossval= CrossValidator(estimator=pipeline3,  
                         estimatorParamMaps=paramgrid3,
                         evaluator=evaluator, 
                         numFolds=10
                        )
#Treinando o modelo e exbindo o score
cvModel3=crossval.fit(training) 
dec_tree = evaluator.evaluate(cvModel3.transform(test))
dec_tree

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.24275118518660851

In [34]:
import numpy as np
import pandas as pd
import pyspark.sql as sparksql

data = np.array([(linear_reg, r_forest_reg, dec_tree)])

modelos = pd.DataFrame(data, columns = ['LinerRegression', 
                                        'RandomForestRegression', 'DecisionTreeRegressor'])

sql_context = sparksql.SQLContext(sc)
df_spark = sql_context.createDataFrame(modelos)
df_spark.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------------+---------------------+
|    LinerRegression|RandomForestRegression|DecisionTreeRegressor|
+-------------------+----------------------+---------------------+
|0.26657931014085273|   0.20084046235790612|  0.24275118518660851|
+-------------------+----------------------+---------------------+